In [1]:
import pandas as pd
df_clean = pd.read_parquet("../data/interim/final_clean.parquet")
df_clean.head()


,row_name,clip_start,clip_end,set_num,set_num:_time,set_p1,set_p1:_time,set_p2,set_p2:_time,servicio,...,golpe_q:_time,fin_golpe,fin_golpe:_time,fin_golpe:_x,fin_golpe:_y,inicio_golpe,inicio_golpe:_time,inicio_golpe:_x,inicio_golpe:_y,__source_file
0,1º Servicio,108.058,108.788,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1º Servicio Arturo Coello,...,108.058,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,25_Roterdam_CoelloTapia_NietoYanguas CSV.csv
1,1º Servicio,131.279,132.01,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1º Servicio Arturo Coello,...,131.279,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,25_Roterdam_CoelloTapia_NietoYanguas CSV.csv
2,1º Servicio,162.04,162.77,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1º Servicio Arturo Coello,...,162.04,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,25_Roterdam_CoelloTapia_NietoYanguas CSV.csv
3,1º Servicio,194.415,195.146,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1º Servicio Arturo Coello,...,194.415,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,25_Roterdam_CoelloTapia_NietoYanguas CSV.csv
4,1º Servicio,214.437,215.167,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1º Servicio Arturo Coello,...,214.437,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,25_Roterdam_CoelloTapia_NietoYanguas CSV.csv


In [10]:
import pandas as pd
import numpy as np

# === 1️⃣ leer y preparar el archivo ===
# Asegúrate de tener df_clean cargado antes
# df_clean = pd.read_excel("limpios_cokiyanguas_ordenados.xlsx")

df = df_clean.copy()
cols = ["clip_start", "juego_p1", "juego_p2", "set_p1", "set_p2"]
df = df[[c for c in cols if c in df.columns]].copy()
df = df.sort_values("clip_start").reset_index(drop=True)

# numéricos
for c in ["juego_p1", "juego_p2", "set_p1", "set_p2"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df["set_inferido_auto"] = False

# === 2️⃣ propagar juegos ===
df["juego_p1"] = df["juego_p1"].ffill().fillna(0).astype(int)
df["juego_p2"] = df["juego_p2"].ffill().fillna(0).astype(int)

# === 3️⃣ corregir sets SOLO si uno está NaN y el otro no ===
for i in range(1, len(df)):
    s1_prev, s2_prev = df.loc[i - 1, ["set_p1", "set_p2"]]
    s1, s2 = df.loc[i, ["set_p1", "set_p2"]]

    if pd.isna(s1) and pd.isna(s2):
        df.loc[i, ["set_p1", "set_p2"]] = [s1_prev, s2_prev]
        continue

    if pd.isna(s1) and pd.notna(s2):
        if s2 in (0, 1):
            df.loc[i, "set_p1"] = 1 - int(s2)
            df.loc[i, "set_inferido_auto"] = True
        else:
            df.loc[i, "set_p1"] = s1_prev
    elif pd.isna(s2) and pd.notna(s1):
        if s1 in (0, 1):
            df.loc[i, "set_p2"] = 1 - int(s1)
            df.loc[i, "set_inferido_auto"] = True
        else:
            df.loc[i, "set_p2"] = s2_prev

df[["set_p1", "set_p2"]] = df[["set_p1", "set_p2"]].ffill().fillna(0).astype(int)

# === 4️⃣ detectar cambios de set ===
df["juego_p1_prev"] = df["juego_p1"].shift(1).fillna(0)
df["juego_p2_prev"] = df["juego_p2"].shift(1).fillna(0)
df["set_p1_prev"] = df["set_p1"].shift(1).fillna(0)
df["set_p2_prev"] = df["set_p2"].shift(1).fillna(0)

cond_set_exp = (df["set_p1"] != df["set_p1_prev"]) | (df["set_p2"] != df["set_p2_prev"])
cond_set_imp = ((df["juego_p1"] == 0) & (df["juego_p2"] == 0)) & (
    (df["juego_p1_prev"] > 0) | (df["juego_p2_prev"] > 0)
)
df["cambio_set"] = cond_set_exp | cond_set_imp

# === 5️⃣ inferir marcador del set cerrado ===
def inferir_y_validar(row):
    p1, p2 = int(row["juego_p1_prev"]), int(row["juego_p2_prev"])
    if p1 == p2 == 0:
        return None
    if p1 > p2:
        f1, f2 = max(6, p1 + 1), p2
    elif p2 > p1:
        f1, f2 = p1, max(6, p2 + 1)
    else:
        return None
    if (
        (f1 >= 6 or f2 >= 6)
        and (
            abs(f1 - f2) >= 2
            or (f1 == 7 and f2 in (5, 6))
            or (f2 == 7 and f1 in (5, 6))
        )
    ):
        return (f1, f2)
    return None

df["set_inferido"] = df.apply(lambda r: inferir_y_validar(r) if r["cambio_set"] else None, axis=1)
df["juegos_finalizados_inferidos"] = df["set_inferido"].apply(lambda x: sum(x) if isinstance(x, tuple) else 0)

# === 6️⃣ acumulados ===
df["juegos_acumulados_sets"] = df["juegos_finalizados_inferidos"].cumsum()
df["progreso_set_actual"] = df["juego_p1"] + df["juego_p2"]
df.loc[df["cambio_set"], "progreso_set_actual"] = 0
df["juegos_totales_acumulados"] = df["juegos_acumulados_sets"] + df["progreso_set_actual"]

# === 7️⃣ log de juegos ===
df["juegos_prev"] = df["juegos_totales_acumulados"].shift(1).fillna(0)
df["marcador_sets_actual"] = df["set_p1"].astype(str) + "-" + df["set_p2"].astype(str)
df["marcador_juegos_actual"] = df["juego_p1"].astype(str) + "-" + df["juego_p2"].astype(str)

df_juegos = df[df["juegos_totales_acumulados"] > df["juegos_prev"]][[
    "clip_start", "set_p1", "set_p2",
    "juego_p1", "juego_p2",
    "marcador_sets_actual", "marcador_juegos_actual",
    "juegos_totales_acumulados", "set_inferido_auto"
]]

# --- 🔧 CORRECCIÓN: eliminar duplicados y saltos anómalos ---
df_juegos = df_juegos.sort_values("clip_start").copy()
df_juegos = df_juegos.loc[
    ~(
        (df_juegos["juegos_totales_acumulados"].diff() > 5)
        & (df_juegos["juegos_totales_acumulados"].diff().shift(-1) < -5)
    )
].drop_duplicates(subset=["clip_start"], keep="first").reset_index(drop=True)
# --- fin corrección ---

print("\n--- PROGRESIÓN DE JUEGOS ---")
print(df_juegos.head(40).to_string(index=False))

# === 8️⃣ RESUMEN DE SETS ROBUSTO + ÚLTIMO SET INFERIDO ===
resumen_sets = []
set_counter = 0

run_s1, run_s2 = 0, 0
last_combo = (0, 0)
sets_rob_p1, sets_rob_p2 = [], []
sets_detalle = []
last_set_recorded_idx = -1

for i, r in df.iterrows():
    s1_raw, s2_raw = r["set_p1"], r["set_p2"]
    set_tuple = r["set_inferido"] if isinstance(r["set_inferido"], tuple) else None
    changed_now = False

    cur_combo = (int(s1_raw), int(s2_raw)) if pd.notna(s1_raw) and pd.notna(s2_raw) else None
    if cur_combo is not None and cur_combo != last_combo:
        run_s1, run_s2 = cur_combo
        last_combo = cur_combo
        changed_now = True

    if (
        bool(r["cambio_set"])
        and set_tuple is not None
        and i > last_set_recorded_idx
    ):
        p1g, p2g = set_tuple

        if not changed_now:
            if p1g > p2g:
                run_s1 += 1
            elif p2g > p1g:
                run_s2 += 1
            last_combo = (run_s1, run_s2)

        if (
            not resumen_sets
            or r["clip_start"] != resumen_sets[-1]["clip_start"]
            or f"{int(p1g)}-{int(p2g)}" != resumen_sets[-1]["Marcador_Set"]
        ):
            set_counter += 1
            ganador = "P1" if p1g > p2g else "P2"
            sets_detalle.append(f"{int(p1g)}-{int(p2g)}")
            resumen_sets.append({
                "Set": set_counter,
                "clip_start": r["clip_start"],
                "Marcador_Set": f"{int(p1g)}-{int(p2g)}",
                "Ganador": ganador,
                "Marcador_Sets_Total": f"{run_s1}-{run_s2}"
            })
            last_set_recorded_idx = i

    sets_rob_p1.append(run_s1)
    sets_rob_p2.append(run_s2)

df["sets_robusto_p1"] = sets_rob_p1
df["sets_robusto_p2"] = sets_rob_p2
df["marcador_sets_robusto"] = df["sets_robusto_p1"].astype(str) + "-" + df["sets_robusto_p2"].astype(str)

# --- inferir el último set si no hay cambio_set final ---
ultima_fila = df.iloc[-1]
if not df["cambio_set"].iloc[-1]:
    j1, j2 = int(ultima_fila["juego_p1"]), int(ultima_fila["juego_p2"])
    if j1 != j2:
        ganador = "P1" if j1 > j2 else "P2"
        if ganador == "P1":
            run_s1 += 1
        else:
            run_s2 += 1
        marcador_final_set = f"{j1+1}-{j2}" if ganador == "P1" else f"{j1}-{j2+1}"
        sets_detalle.append(marcador_final_set)
        set_counter += 1
        resumen_sets.append({
            "Set": set_counter,
            "clip_start": ultima_fila["clip_start"],
            "Marcador_Set": marcador_final_set,
            "Ganador": ganador,
            "Marcador_Sets_Total": f"{run_s1}-{run_s2}"
        })

# --- RESUMEN FINAL ---
print("\n--- RESUMEN DE SETS ---")
df_resumen = pd.DataFrame(resumen_sets)
if not df_resumen.empty:
    print(df_resumen)
else:
    print("(No se detectaron sets cerrados con marcador válido)")

if sets_detalle:
    marcador_detalle = ", ".join(sets_detalle)
    marcador_total = f"{run_s1}-{run_s2}"
    print(f"\n🏁 Marcador final: {marcador_detalle}")
    print(f"Marcador total en sets: {marcador_total}")
else:
    print("\n(No se detectaron sets cerrados con marcador válido)")

# === 🔟 recorte por límite de juegos ===
def recortar_por_limite(df, limite_juegos, resumen_sets):
    mask = df["juegos_totales_acumulados"] >= limite_juegos

    if mask.any():
        idx = mask.idxmax()
        fila = df.loc[idx]
        print(f"\n✅ Se alcanzó el límite de {limite_juegos} juegos en la fila {idx}.")

        resumen_prev = resumen_sets.loc[resumen_sets["clip_start"] <= fila["clip_start"]]
        marcadores_cerrados = resumen_prev["Marcador_Set"].tolist() if not resumen_prev.empty else []

        idx_set_inicio = df.loc[:idx, "cambio_set"][::-1].idxmax() if df.loc[:idx, "cambio_set"].any() else 0
        df_set_actual = df.loc[idx_set_inicio:idx]
        j1_actual = int(df_set_actual["juego_p1"].iloc[-1])
        j2_actual = int(df_set_actual["juego_p2"].iloc[-1])
        marcador_set_actual = f"{j1_actual}-{j2_actual}"

        marcador_completo = " ".join(marcadores_cerrados + [marcador_set_actual])
    else:
        fila = df.iloc[-1]
        print(f"\n⚠️ No se alcanzó el límite de {limite_juegos} juegos (final del partido).")
        marcadores_cerrados = resumen_sets["Marcador_Set"].tolist() if not resumen_sets.empty else []
        marcador_completo = " ".join(marcadores_cerrados)

    print(f"⏱ clip_start: {fila['clip_start']}")
    print(f"Marcador actual (por juegos): {marcador_completo}")
    print(f"Juegos totales acumulados: {int(fila['juegos_totales_acumulados'])}")
    return fila

# --- ejemplo ---
LIMITE_JUEGOS = 11
if 'df_resumen' not in locals():
    df_resumen = pd.DataFrame(columns=["Set", "clip_start", "Marcador_Set", "Ganador", "Marcador_Sets_Total"])

fila_objetivo = recortar_por_limite(df, LIMITE_JUEGOS, df_resumen)



--- PROGRESIÓN DE JUEGOS ---
 clip_start  set_p1  set_p2  juego_p1  juego_p2 marcador_sets_actual marcador_juegos_actual  juegos_totales_acumulados  set_inferido_auto
    282.127       0       0         1         0                  0-0                    1-0                          1              False
    361.927       0       0         1         1                  0-0                    1-1                          2              False
    578.704       0       0         2         1                  0-0                    2-1                          3              False
     729.19       0       0         2         2                  0-0                    2-2                          4              False
    991.523       0       0         3         2                  0-0                    3-2                          5              False
   1402.515       0       0         3         3                  0-0                    3-3                          6              False
    

In [19]:
import pandas as pd
import numpy as np

# === 1️⃣ leer y preparar el archivo ===
# Asegúrate de tener df_clean cargado antes
# df_clean = pd.read_excel("limpios_cokiyanguas_ordenados.xlsx")

df = df_clean.copy()
cols = ["clip_start", "juego_p1", "juego_p2", "set_p1", "set_p2"]
df = df[[c for c in cols if c in df.columns]].copy()
df = df.sort_values("clip_start").reset_index(drop=True)

# numéricos
for c in ["juego_p1", "juego_p2", "set_p1", "set_p2"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df["set_inferido_auto"] = False

# === 2️⃣ propagar juegos ===
df["juego_p1"] = df["juego_p1"].ffill().fillna(0).astype(int)
df["juego_p2"] = df["juego_p2"].ffill().fillna(0).astype(int)

# === 3️⃣ corregir sets SOLO si uno está NaN y el otro no ===
for i in range(1, len(df)):
    s1_prev, s2_prev = df.loc[i - 1, ["set_p1", "set_p2"]]
    s1, s2 = df.loc[i, ["set_p1", "set_p2"]]

    if pd.isna(s1) and pd.isna(s2):
        df.loc[i, ["set_p1", "set_p2"]] = [s1_prev, s2_prev]
        continue

    if pd.isna(s1) and pd.notna(s2):
        if s2 in (0, 1):
            df.loc[i, "set_p1"] = 1 - int(s2)
            df.loc[i, "set_inferido_auto"] = True
        else:
            df.loc[i, "set_p1"] = s1_prev
    elif pd.isna(s2) and pd.notna(s1):
        if s1 in (0, 1):
            df.loc[i, "set_p2"] = 1 - int(s1)
            df.loc[i, "set_inferido_auto"] = True
        else:
            df.loc[i, "set_p2"] = s2_prev

df[["set_p1", "set_p2"]] = df[["set_p1", "set_p2"]].ffill().fillna(0).astype(int)

# === 4️⃣ detectar cambios de set ===
df["juego_p1_prev"] = df["juego_p1"].shift(1).fillna(0)
df["juego_p2_prev"] = df["juego_p2"].shift(1).fillna(0)
df["set_p1_prev"] = df["set_p1"].shift(1).fillna(0)
df["set_p2_prev"] = df["set_p2"].shift(1).fillna(0)

cond_set_exp = (df["set_p1"] != df["set_p1_prev"]) | (df["set_p2"] != df["set_p2_prev"])
cond_set_imp = ((df["juego_p1"] == 0) & (df["juego_p2"] == 0)) & (
    (df["juego_p1_prev"] > 0) | (df["juego_p2_prev"] > 0)
)
df["cambio_set"] = cond_set_exp | cond_set_imp

# === 5️⃣ inferir marcador del set cerrado ===
def inferir_y_validar(row):
    p1, p2 = int(row["juego_p1_prev"]), int(row["juego_p2_prev"])
    if p1 == p2 == 0:
        return None
    if p1 > p2:
        f1, f2 = max(6, p1 + 1), p2
    elif p2 > p1:
        f1, f2 = p1, max(6, p2 + 1)
    else:
        return None
    if (
        (f1 >= 6 or f2 >= 6)
        and (
            abs(f1 - f2) >= 2
            or (f1 == 7 and f2 in (5, 6))
            or (f2 == 7 and f1 in (5, 6))
        )
    ):
        return (f1, f2)
    return None

df["set_inferido"] = df.apply(lambda r: inferir_y_validar(r) if r["cambio_set"] else None, axis=1)
df["juegos_finalizados_inferidos"] = df["set_inferido"].apply(lambda x: sum(x) if isinstance(x, tuple) else 0)

# === 6️⃣ acumulados (versión final definitiva) ===
df["juegos_acumulados_sets"] = df["juegos_finalizados_inferidos"].cumsum()
df["progreso_set_actual"] = df["juego_p1"] + df["juego_p2"]
df.loc[df["cambio_set"], "progreso_set_actual"] = 0

# Contador de juegos lineal, empezando en 0
df["juegos_totales_acumulados"] = 0
contador = 0
for i in range(len(df)):
    if i > 0:
        j_prev = f"{df.loc[i-1, 'juego_p1']}-{df.loc[i-1, 'juego_p2']}"
        j_cur = f"{df.loc[i, 'juego_p1']}-{df.loc[i, 'juego_p2']}"
        s_prev = f"{df.loc[i-1, 'set_p1']}-{df.loc[i-1, 'set_p2']}"
        s_cur = f"{df.loc[i, 'set_p1']}-{df.loc[i, 'set_p2']}"
        
        # Sumar +1 solo cuando hay cambio de marcador real (no reinicio 0-0)
        if (j_cur != j_prev) and not (j_cur == "0-0" and s_cur != s_prev):
            contador += 1
    df.loc[i, "juegos_totales_acumulados"] = contador



# === 7️⃣ log de juegos ===
df["juegos_prev"] = df["juegos_totales_acumulados"].shift(1).fillna(0)
df["marcador_sets_actual"] = df["set_p1"].astype(str) + "-" + df["set_p2"].astype(str)
df["marcador_juegos_actual"] = df["juego_p1"].astype(str) + "-" + df["juego_p2"].astype(str)

df_juegos = df[df["juegos_totales_acumulados"] > df["juegos_prev"]][[
    "clip_start", "set_p1", "set_p2",
    "juego_p1", "juego_p2",
    "marcador_sets_actual", "marcador_juegos_actual",
    "juegos_totales_acumulados", "set_inferido_auto"
]]

# --- 🔧 CORRECCIÓN DEFINITIVA: eliminar fila duplicada del cierre de set ---
df_juegos = df_juegos.sort_values("clip_start").reset_index(drop=True)

mask_dup_cierre = (
    (df_juegos["set_p1"].diff() != 0) | (df_juegos["set_p2"].diff() != 0)
) & (
    df_juegos["marcador_juegos_actual"] == df_juegos["marcador_juegos_actual"].shift(1)
)

df_juegos = df_juegos.loc[~mask_dup_cierre].reset_index(drop=True)
# --- fin corrección ---

print("\n--- PROGRESIÓN DE JUEGOS ---")
print(df_juegos.head(40).to_string(index=False))

# === 8️⃣ RESUMEN DE SETS ROBUSTO + ÚLTIMO SET INFERIDO ===
resumen_sets = []
set_counter = 0

run_s1, run_s2 = 0, 0
last_combo = (0, 0)
sets_rob_p1, sets_rob_p2 = [], []
sets_detalle = []
last_set_recorded_idx = -1

for i, r in df.iterrows():
    s1_raw, s2_raw = r["set_p1"], r["set_p2"]
    set_tuple = r["set_inferido"] if isinstance(r["set_inferido"], tuple) else None
    changed_now = False

    cur_combo = (int(s1_raw), int(s2_raw)) if pd.notna(s1_raw) and pd.notna(s2_raw) else None
    if cur_combo is not None and cur_combo != last_combo:
        run_s1, run_s2 = cur_combo
        last_combo = cur_combo
        changed_now = True

    if (
        bool(r["cambio_set"])
        and set_tuple is not None
        and i > last_set_recorded_idx
    ):
        p1g, p2g = set_tuple

        if not changed_now:
            if p1g > p2g:
                run_s1 += 1
            elif p2g > p1g:
                run_s2 += 1
            last_combo = (run_s1, run_s2)

        if (
            not resumen_sets
            or r["clip_start"] != resumen_sets[-1]["clip_start"]
            or f"{int(p1g)}-{int(p2g)}" != resumen_sets[-1]["Marcador_Set"]
        ):
            set_counter += 1
            ganador = "P1" if p1g > p2g else "P2"
            sets_detalle.append(f"{int(p1g)}-{int(p2g)}")
            resumen_sets.append({
                "Set": set_counter,
                "clip_start": r["clip_start"],
                "Marcador_Set": f"{int(p1g)}-{int(p2g)}",
                "Ganador": ganador,
                "Marcador_Sets_Total": f"{run_s1}-{run_s2}"
            })
            last_set_recorded_idx = i

    sets_rob_p1.append(run_s1)
    sets_rob_p2.append(run_s2)

df["sets_robusto_p1"] = sets_rob_p1
df["sets_robusto_p2"] = sets_rob_p2
df["marcador_sets_robusto"] = df["sets_robusto_p1"].astype(str) + "-" + df["sets_robusto_p2"].astype(str)

# --- inferir el último set si no hay cambio_set final ---
ultima_fila = df.iloc[-1]
if not df["cambio_set"].iloc[-1]:
    j1, j2 = int(ultima_fila["juego_p1"]), int(ultima_fila["juego_p2"])
    if j1 != j2:
        ganador = "P1" if j1 > j2 else "P2"
        if ganador == "P1":
            run_s1 += 1
        else:
            run_s2 += 1
        marcador_final_set = f"{j1+1}-{j2}" if ganador == "P1" else f"{j1}-{j2+1}"
        sets_detalle.append(marcador_final_set)
        set_counter += 1
        resumen_sets.append({
            "Set": set_counter,
            "clip_start": ultima_fila["clip_start"],
            "Marcador_Set": marcador_final_set,
            "Ganador": ganador,
            "Marcador_Sets_Total": f"{run_s1}-{run_s2}"
        })

# --- RESUMEN FINAL ---
print("\n--- RESUMEN DE SETS ---")
df_resumen = pd.DataFrame(resumen_sets)
if not df_resumen.empty:
    print(df_resumen)
else:
    print("(No se detectaron sets cerrados con marcador válido)")

if sets_detalle:
    marcador_detalle = ", ".join(sets_detalle)
    marcador_total = f"{run_s1}-{run_s2}"
    print(f"\n🏁 Marcador final: {marcador_detalle}")
    print(f"Marcador total en sets: {marcador_total}")
else:
    print("\n(No se detectaron sets cerrados con marcador válido)")

# === 🔟 recorte por límite de juegos ===
def recortar_por_limite(df, limite_juegos, resumen_sets):
    mask = df["juegos_totales_acumulados"] >= limite_juegos

    if mask.any():
        idx = mask.idxmax()
        fila = df.loc[idx]
        print(f"\n✅ Se alcanzó el límite de {limite_juegos} juegos en la fila {idx}.")

        # sets cerrados hasta ese punto
        resumen_prev = resumen_sets.loc[resumen_sets["clip_start"] <= fila["clip_start"]]
        marcadores_cerrados = resumen_prev["Marcador_Set"].tolist() if not resumen_prev.empty else []

        # calcular marcador parcial dentro del set actual
        idx_set_inicio = df.loc[:idx, "cambio_set"][::-1].idxmax() if df.loc[:idx, "cambio_set"].any() else 0
        df_set_actual = df.loc[idx_set_inicio:idx]

        j1_actual = int(df_set_actual["juego_p1"].iloc[-1])
        j2_actual = int(df_set_actual["juego_p2"].iloc[-1])
        marcador_set_actual = f"{j1_actual}-{j2_actual}"

        # marcador final parcial correcto (p. ej. 4-6 6-1 2-0)
        marcador_completo = " ".join(marcadores_cerrados + [marcador_set_actual])
    else:
        fila = df.iloc[-1]
        print(f"\n⚠️ No se alcanzó el límite de {limite_juegos} juegos (final del partido).")
        marcadores_cerrados = resumen_sets["Marcador_Set"].tolist() if not resumen_sets.empty else []
        marcador_completo = " ".join(marcadores_cerrados)

    print(f"⏱ clip_start: {fila['clip_start']}")
    print(f"Marcador actual (por juegos): {marcador_completo}")
    print(f"Juegos totales acumulados: {int(fila['juegos_totales_acumulados'])}")
    return fila


# --- ejemplo ---
LIMITE_JUEGOS = 23
if 'df_resumen' not in locals():
    df_resumen = pd.DataFrame(columns=["Set", "clip_start", "Marcador_Set", "Ganador", "Marcador_Sets_Total"])

fila_objetivo = recortar_por_limite(df, LIMITE_JUEGOS, df_resumen)



--- PROGRESIÓN DE JUEGOS ---
 clip_start  set_p1  set_p2  juego_p1  juego_p2 marcador_sets_actual marcador_juegos_actual  juegos_totales_acumulados  set_inferido_auto
    282.127       0       0         1         0                  0-0                    1-0                          1              False
    361.927       0       0         1         1                  0-0                    1-1                          2              False
    578.704       0       0         2         1                  0-0                    2-1                          3              False
     729.19       0       0         2         2                  0-0                    2-2                          4              False
    991.523       0       0         3         2                  0-0                    3-2                          5              False
   1402.515       0       0         3         3                  0-0                    3-3                          6              False
    